In [1]:
#Import dependencies
import pandas as pd
import requests

#API pull for latest salary info from www.levels.fyi
salaryData = requests.get('https://www.levels.fyi/js/salaryData.json').json()
salary_df = pd.DataFrame(salaryData)

In [2]:
#raw table preview
salary_df.head(10)

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,otherdetails,cityid,dmaid,rowNumber
0,6/7/2017 11:33:27,Oracle,L3,Product Manager,127,"Redwood City, CA",1.5,1.5,,107,20,10,,,7392,807,1
1,6/10/2017 17:11:29,eBay,SE 2,Software Engineer,100,"San Francisco, CA",5,3,,,,,,,7419,807,2
2,6/11/2017 14:53:57,Amazon,L7,Product Manager,310,"Seattle, WA",8,0,,155,,,,,11527,819,3
3,6/14/2017 21:22:25,Microsoft,64,Software Engineering Manager,200,"Redmond, WA",9,9,,169000,100000,30000,,,11521,819,5
4,6/16/2017 10:44:01,Amazon,L5,Software Engineer,173,"Vancouver, BC, Canada",11,1,,120000,0,53000,,,1320,0,6
5,6/17/2017 0:23:14,Apple,M1,Software Engineering Manager,372,"Sunnyvale, CA",7,5,,157,180,35,,,7472,807,7
6,6/20/2017 10:58:51,Microsoft,60,Software Engineer,157,"Mountain View, CA",5,3,,,,,,,7322,807,9
7,6/20/2017 18:49:59,Amazon,L5,Software Engineer,190,"Seattle, WA",3,3,,110000,80000,,,,11527,819,10
8,6/21/2017 17:27:47,Microsoft,63,Software Engineer,208,"Seattle, WA",8.5,8.5,,,,,,,11527,819,11
9,6/22/2017 12:37:51,Microsoft,65,Software Engineering Manager,300,"Redmond, WA",15,11,,180,65,55,,,11521,819,12


In [3]:
#dropping columns that are not relevant to project
salary_df = salary_df.drop(['cityid', 'dmaid','rowNumber','otherdetails',], axis=1)
salary_df.head(10)

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender
0,6/7/2017 11:33:27,Oracle,L3,Product Manager,127,"Redwood City, CA",1.5,1.5,,107,20,10,
1,6/10/2017 17:11:29,eBay,SE 2,Software Engineer,100,"San Francisco, CA",5,3,,,,,
2,6/11/2017 14:53:57,Amazon,L7,Product Manager,310,"Seattle, WA",8,0,,155,,,
3,6/14/2017 21:22:25,Microsoft,64,Software Engineering Manager,200,"Redmond, WA",9,9,,169000,100000,30000,
4,6/16/2017 10:44:01,Amazon,L5,Software Engineer,173,"Vancouver, BC, Canada",11,1,,120000,0,53000,
5,6/17/2017 0:23:14,Apple,M1,Software Engineering Manager,372,"Sunnyvale, CA",7,5,,157,180,35,
6,6/20/2017 10:58:51,Microsoft,60,Software Engineer,157,"Mountain View, CA",5,3,,,,,
7,6/20/2017 18:49:59,Amazon,L5,Software Engineer,190,"Seattle, WA",3,3,,110000,80000,,
8,6/21/2017 17:27:47,Microsoft,63,Software Engineer,208,"Seattle, WA",8.5,8.5,,,,,
9,6/22/2017 12:37:51,Microsoft,65,Software Engineering Manager,300,"Redmond, WA",15,11,,180,65,55,


In [4]:
#converting to float to allow for summary stats
salary_df["totalyearlycompensation"] = pd.to_numeric(salary_df["totalyearlycompensation"])
salary_df["yearsofexperience"] = pd.to_numeric(salary_df["yearsofexperience"])
salary_df["yearsatcompany"] = pd.to_numeric(salary_df["yearsatcompany"])
salary_df["basesalary"] = pd.to_numeric(salary_df["basesalary"])
salary_df["stockgrantvalue"] = pd.to_numeric(salary_df["stockgrantvalue"])
salary_df["bonus"] = pd.to_numeric(salary_df["bonus"])

#coverting timestamp from object to datetime
salary_df['timestamp'] =  pd.to_datetime(salary_df['timestamp'], infer_datetime_format=True)

In [5]:
# Create separate cols for city, state and country
def split_location(location):
    items = location.split(', ')
    city = items[0]
    state = items[1]
    
    if len(items)==2:
        country = 'US'
    elif len(items)==3:
        country = items[2].strip()
    elif len(items)==4:
        country = ', '.join([i.strip() for i in items[2:]])
    else:
        country = None
        print(location)
        
    return [city, state, country]



In [6]:
salary_df['loc_items'] = salary_df.location.apply(lambda x: split_location(x))
salary_df['city'] = salary_df.loc_items.apply(lambda x: x[0])
salary_df['state'] = salary_df.loc_items.apply(lambda x: x[1])
salary_df['country'] = salary_df.loc_items.apply(lambda x: x[2])

In [7]:
salary_df.head()

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,loc_items,city,state,country
0,2017-06-07 11:33:27,Oracle,L3,Product Manager,127.0,"Redwood City, CA",1.5,1.5,,107.0,20.0,10.0,,"[Redwood City, CA, US]",Redwood City,CA,US
1,2017-06-10 17:11:29,eBay,SE 2,Software Engineer,100.0,"San Francisco, CA",5.0,3.0,,NaN,NaN,NaN,,"[San Francisco, CA, US]",San Francisco,CA,US
2,2017-06-11 14:53:57,Amazon,L7,Product Manager,310.0,"Seattle, WA",8.0,0.0,,155.0,NaN,NaN,,"[Seattle, WA, US]",Seattle,WA,US
3,2017-06-14 21:22:25,Microsoft,64,Software Engineering Manager,200.0,"Redmond, WA",9.0,9.0,,169000.0,100000.0,30000.0,,"[Redmond, WA, US]",Redmond,WA,US
4,2017-06-16 10:44:01,Amazon,L5,Software Engineer,173.0,"Vancouver, BC, Canada",11.0,1.0,,120000.0,0.0,53000.0,,"[Vancouver, BC, Canada]",Vancouver,BC,Canada


In [8]:
# droping location column  
salary_df = salary_df.drop(['location','loc_items'], axis=1)
salary_df.head(10)

,timestamp,company,level,title,totalyearlycompensation,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,city,state,country
0,2017-06-07 11:33:27,Oracle,L3,Product Manager,127.0,1.5,1.5,,107.0,20.0,10.0,,Redwood City,CA,US
1,2017-06-10 17:11:29,eBay,SE 2,Software Engineer,100.0,5.0,3.0,,NaN,NaN,NaN,,San Francisco,CA,US
2,2017-06-11 14:53:57,Amazon,L7,Product Manager,310.0,8.0,0.0,,155.0,NaN,NaN,,Seattle,WA,US
3,2017-06-14 21:22:25,Microsoft,64,Software Engineering Manager,200.0,9.0,9.0,,169000.0,100000.0,30000.0,,Redmond,WA,US
4,2017-06-16 10:44:01,Amazon,L5,Software Engineer,173.0,11.0,1.0,,120000.0,0.0,53000.0,,Vancouver,BC,Canada
5,2017-06-17 00:23:14,Apple,M1,Software Engineering Manager,372.0,7.0,5.0,,157.0,180.0,35.0,,Sunnyvale,CA,US
6,2017-06-20 10:58:51,Microsoft,60,Software Engineer,157.0,5.0,3.0,,NaN,NaN,NaN,,Mountain View,CA,US
7,2017-06-20 18:49:59,Amazon,L5,Software Engineer,190.0,3.0,3.0,,110000.0,80000.0,NaN,,Seattle,WA,US
8,2017-06-21 17:27:47,Microsoft,63,Software Engineer,208.0,8.5,8.5,,NaN,NaN,NaN,,Seattle,WA,US
9,2017-06-22 12:37:51,Microsoft,65,Software Engineering Manager,300.0,15.0,11.0,,180.0,65.0,55.0,,Redmond,WA,US


In [9]:
#isolating US data for further exploration
us_df = salary_df[salary_df.country=='US'].copy()

In [10]:
#isolating us data to data scientist titles
us_df = us_df[us_df.title=='Data Scientist'].copy()
us_df

,timestamp,company,level,title,totalyearlycompensation,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,city,state,country
745,2018-06-05 14:06:30,LinkedIn,Senior,Data Scientist,233.0,4.0,0.0,Data Analysis,162.0,220.0,10.0,Male,San Francisco,CA,US
772,2018-06-08 00:29:47,Amazon,L4,Data Scientist,140.0,2.0,2.0,,92000.0,48000.0,NaN,Male,Seattle,WA,US
776,2018-06-08 09:49:25,Microsoft,64,Data Scientist,218.0,11.0,11.0,ML / AI,165.0,28.0,23.0,Male,Seattle,WA,US
782,2018-06-08 17:55:09,ebay,26,Data Scientist,180.0,10.0,5.0,,NaN,NaN,NaN,Female,San Jose,CA,US
796,2018-06-10 19:39:35,Twitter,Staff,Data Scientist,500.0,4.0,4.0,ML / AI,200.0,280.0,20.0,Male,San Francisco,CA,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62569,2021-08-16 16:17:19,IBM,L5,Data Scientist,145.0,6.0,5.0,Data,140.0,0.0,5.0,Male,New City,NY,US
62578,2021-08-16 17:08:58,Booz Allen Hamilton,Senior Consultant,Data Scientist,110.0,0.0,0.0,General,110.0,0.0,0.0,Male,West McLean,VA,US
62600,2021-08-16 21:02:37,Xandr,L1,Data Scientist,120.0,1.0,0.0,General,110.0,0.0,10.0,Male,Portland,OR,US
62610,2021-08-16 22:19:48,Facebook,L4,Data Scientist,233.0,2.0,2.0,Data Engineering,157.0,60.0,16.0,Male,Menlo Park,CA,US


In [11]:
#exploring us/datascience/IL at amazon
google_df = us_df[us_df.company=='Google'].copy()
google_df

,timestamp,company,level,title,totalyearlycompensation,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,city,state,country
822,2018-06-12 20:54:06,Google,T6,Data Scientist,685.0,22.0,2.0,ML / AI,221000.0,296000.0,55000.0,Male,Kirkland,WA,US
877,2018-06-20 00:49:11,Google,L6,Data Scientist,690.0,10.0,0.0,ML / AI,240.0,298.0,72.0,Male,Kirkland,WA,US
1820,2018-09-14 00:29:43,Google,L6,Data Scientist,700.0,6.0,2.0,ML / AI,221.0,300.0,44.0,,Seattle,WA,US
2043,2018-09-26 22:14:28,Google,L4,Data Scientist,210.0,1.0,1.0,Full Stack,132.0,42.0,36.0,,Mountain View,CA,US
2044,2018-09-26 22:15:36,Google,L4,Data Scientist,210.0,1.0,1.0,Full Stack,132.0,42.0,36.0,,Mountain View,CA,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56490,2021-06-26 08:00:49,Google,L3,Data Scientist,197.0,3.0,2.0,General,121.0,55.0,22.0,Male,San Francisco,CA,US
57070,2021-07-01 07:00:01,Google,L3,Data Scientist,250.0,1.0,1.0,Product,175.0,40.0,35.0,,San Francisco,CA,US
58916,2021-07-18 16:55:05,Google,L3,Data Scientist,175.0,2.0,1.0,data analyst,150.0,20.0,5.0,,San Francisco,CA,US
59338,2021-07-21 22:29:24,Google,L5,Data Scientist,225.0,10.0,2.0,commerce,174.0,19.0,32.0,Male,Mountain View,CA,US


In [12]:
#exploring targeted results for amazon levels reporting
google_df.level.value_counts()

L4                   36
L5                   22
L3                    9
L6                    7
5                     2
L2                    1
L7                    1
L1                    1
Product Manager 3     1
T6                    1
Data                  1
Name: level, dtype: int64

In [13]:
#comparing medians for insights on compareable levels
google_levels_df = google_df.groupby(['level']).agg({'totalyearlycompensation':'median', 'basesalary':'median', 'yearsofexperience':'median', 'yearsatcompany':'median',
                                                'stockgrantvalue':'median', 'bonus':'median'  })
google_levels_df

,totalyearlycompensation,basesalary,yearsofexperience,yearsatcompany,stockgrantvalue,bonus
level,,,,,,
5,292.5,165.0,9.0,3.5,99.0,28.5
Data,120.0,100.0,2.0,1.0,12.0,10.0
L1,100.0,70.0,0.0,0.0,20.0,10.0
L2,120.0,NaN,1.0,0.0,NaN,NaN
L3,175.0,121.0,2.0,1.0,40.0,20.0
L4,220.0,143.5,3.5,2.0,56.5,25.0
L5,279.0,171.5,7.5,2.0,86.5,29.0
L6,570.0,224.5,10.0,3.0,299.0,47.0
L7,460.0,235.0,10.0,4.0,165.0,60.0


In [14]:
#Correcting 5 to L5 as they have similar basesalary/bonus
google_df.replace('5', 'L5', inplace=True)

In [15]:
#exploring targeted results for amazon levels reporting
google_df.level.value_counts()

L4                   36
L5                   24
L3                    9
L6                    7
L2                    1
L7                    1
L1                    1
Product Manager 3     1
T6                    1
Data                  1
Name: level, dtype: int64

In [16]:
#comparing medians for insights on compareable levels
google_levels_df = google_df.groupby(['level']).agg({'totalyearlycompensation':'median', 'basesalary':'median', 'yearsofexperience':'median', 'yearsatcompany':'median',
                                                'stockgrantvalue':'median', 'bonus':'median'  })
google_levels_df

,totalyearlycompensation,basesalary,yearsofexperience,yearsatcompany,stockgrantvalue,bonus
level,,,,,,
Data,120.0,100.0,2.0,1.0,12.0,10.0
L1,100.0,70.0,0.0,0.0,20.0,10.0
L2,120.0,NaN,1.0,0.0,NaN,NaN
L3,175.0,121.0,2.0,1.0,40.0,20.0
L4,220.0,143.5,3.5,2.0,56.5,25.0
L5,279.0,170.0,8.0,2.0,86.5,29.0
L6,570.0,224.5,10.0,3.0,299.0,47.0
L7,460.0,235.0,10.0,4.0,165.0,60.0
Product Manager 3,388.0,278.0,0.0,0.0,80.0,30.0


In [17]:
google_df.level.value_counts()

L4                   36
L5                   24
L3                    9
L6                    7
L2                    1
L7                    1
L1                    1
Product Manager 3     1
T6                    1
Data                  1
Name: level, dtype: int64

In [18]:
#removed T6 for extreme values and product manager 3
google_df = google_df.drop(index=google_df[google_df['level'] == 'T6'].index)
google_df = google_df.drop(index=google_df[google_df['level'] == 'Product Manager 3'].index)
#removed L7 and L6 due to high stock value as it indictates upper manangement
google_df = google_df.drop(index=google_df[google_df['level'] == 'L6'].index)
google_df = google_df.drop(index=google_df[google_df['level'] == 'L7'].index)

In [20]:
#comparing medians for insights on compareable levels
google_levels_df = google_df.groupby(['level']).agg({'totalyearlycompensation':'median', 'basesalary':'median', 'yearsofexperience':'median', 'yearsatcompany':'median',
                                                'stockgrantvalue':'median', 'bonus':'median'  })
google_levels_df

,totalyearlycompensation,basesalary,yearsofexperience,yearsatcompany,stockgrantvalue,bonus
level,,,,,,
Data,120.0,100.0,2.0,1.0,12.0,10.0
L1,100.0,70.0,0.0,0.0,20.0,10.0
L2,120.0,NaN,1.0,0.0,NaN,NaN
L3,175.0,121.0,2.0,1.0,40.0,20.0
L4,220.0,143.5,3.5,2.0,56.5,25.0
L5,279.0,170.0,8.0,2.0,86.5,29.0


In [21]:
#removed Data single value due to not having similar yearlycompensation/basesalary/stockgrantvalue/bonus
google_df = google_df.drop(index=google_df[google_df['level'] == 'Data'].index)
#removed L1 single value due to not having similar yearlycompensation/basesalary/stockgrantvalue/bonus
google_df = google_df.drop(index=google_df[google_df['level'] == 'L1'].index)
#removed L2 single value due to not having similar yearlycompensation and missing basesalary/stockgrantvalue/bonus
google_df = google_df.drop(index=google_df[google_df['level'] == 'L2'].index)

In [22]:
#placing each level in our categories based on similiar: basesalary/yearsofexperience/yearsatcompany/stockgrantvalue/bonus
google_df.loc[google_df.level == "L3", "Category"] = "Entry"
google_df.loc[google_df.level == "L4", "Category"] = "Mid"
google_df.loc[google_df.level == "L5", "Category"] = "Senior"

In [23]:
google_df

,timestamp,company,level,title,totalyearlycompensation,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,city,state,country,Category
2043,2018-09-26 22:14:28,Google,L4,Data Scientist,210.0,1.0,1.0,Full Stack,132.0,42.0,36.0,,Mountain View,CA,US,Mid
2044,2018-09-26 22:15:36,Google,L4,Data Scientist,210.0,1.0,1.0,Full Stack,132.0,42.0,36.0,,Mountain View,CA,US,Mid
3760,2018-12-12 22:21:31,Google,L5,Data Scientist,250.0,6.0,6.0,DS,NaN,NaN,NaN,,Mountain View,CA,US,Senior
3761,2018-12-12 22:21:31,Google,L5,Data Scientist,250.0,6.0,6.0,DS,NaN,NaN,NaN,,Mountain View,CA,US,Senior
5841,2019-03-24 15:03:17,Google,L4,Data Scientist,200.0,0.0,0.0,ML / AI,NaN,NaN,NaN,Female,Mountain View,CA,US,Mid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56490,2021-06-26 08:00:49,Google,L3,Data Scientist,197.0,3.0,2.0,General,121.0,55.0,22.0,Male,San Francisco,CA,US,Entry
57070,2021-07-01 07:00:01,Google,L3,Data Scientist,250.0,1.0,1.0,Product,175.0,40.0,35.0,,San Francisco,CA,US,Entry
58916,2021-07-18 16:55:05,Google,L3,Data Scientist,175.0,2.0,1.0,data analyst,150.0,20.0,5.0,,San Francisco,CA,US,Entry
59338,2021-07-21 22:29:24,Google,L5,Data Scientist,225.0,10.0,2.0,commerce,174.0,19.0,32.0,Male,Mountain View,CA,US,Senior
